# **<u>Cleaning the total traffic data:</u>**

In [1]:
import pandas as pd

## **<u>Loading the data using Pandas:</u>**

In [205]:
df = pd.read_csv('../external_data/comptage-multimodal-comptages.csv', sep=';')

In [206]:
df.shape

(556776, 10)

In [207]:
df.head()

,Identifiant trajectoire,Identifiant site,Nom du site,Date et heure de comptage,Mode déplacement,Nombre de véhicules,Type voie,Sens de circulation,Trajectoire,Coordonnées Géo
0,10004_2 -> 1,10004,[Paris] Rivoli x Nicolas Flamel,2021-08-17T02:00:00+02:00,Vélos,6,Piste cyclable,E-O,2 -> 1,"48.858273, 2.349109"
1,10004_5 -> 3,10004,[Paris] Rivoli x Nicolas Flamel,2021-08-17T02:00:00+02:00,Vélos,3,Voie de circulation générale,E-O,5 -> 3,"48.858273, 2.349109"
2,10004_5 -> 3,10004,[Paris] Rivoli x Nicolas Flamel,2021-08-17T03:00:00+02:00,2 roues motorisées,4,Voie de circulation générale,E-O,5 -> 3,"48.858273, 2.349109"
3,10004_2 -> 1,10004,[Paris] Rivoli x Nicolas Flamel,2021-08-17T04:00:00+02:00,Vélos,5,Piste cyclable,E-O,2 -> 1,"48.858273, 2.349109"
4,10004_4 -> 2,10004,[Paris] Rivoli x Nicolas Flamel,2021-08-17T04:00:00+02:00,Vélos,4,Coronapiste,E-O,4 -> 2,"48.858273, 2.349109"


In [208]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 556776 entries, 0 to 556775
Data columns (total 10 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   Identifiant trajectoire    556776 non-null  object
 1   Identifiant site           556776 non-null  int64 
 2   Nom du site                553760 non-null  object
 3   Date et heure de comptage  556776 non-null  object
 4   Mode déplacement           556776 non-null  object
 5   Nombre de véhicules        556776 non-null  int64 
 6   Type voie                  553760 non-null  object
 7   Sens de circulation        553760 non-null  object
 8   Trajectoire                556776 non-null  object
 9   Coordonnées Géo            553760 non-null  object
dtypes: int64(2), object(8)
memory usage: 42.5+ MB


## **<u>Dropping redundant columns:</u>**

In [209]:
df = df.drop(['Identifiant trajectoire', 'Identifiant site', 
         'Nom du site', 'Coordonnées Géo',
         'Mode déplacement', 'Sens de circulation',
         'Trajectoire', 'Type voie'], axis=1) #'Date et heure de comptage',

In [210]:
df

,Date et heure de comptage,Nombre de véhicules
0,2021-08-17T02:00:00+02:00,6
1,2021-08-17T02:00:00+02:00,3
2,2021-08-17T03:00:00+02:00,4
3,2021-08-17T04:00:00+02:00,5
4,2021-08-17T04:00:00+02:00,4
...,...,...
556771,2021-03-29T09:00:00+02:00,1
556772,2021-03-29T14:00:00+02:00,2
556773,2021-03-29T18:00:00+02:00,200
556774,2021-03-29T19:00:00+02:00,4


In [211]:
df['Date et heure de comptage'] = pd.to_datetime(df['Date et heure de comptage'], utc=True)

In [212]:
df

,Date et heure de comptage,Nombre de véhicules
0,2021-08-17 00:00:00+00:00,6
1,2021-08-17 00:00:00+00:00,3
2,2021-08-17 01:00:00+00:00,4
3,2021-08-17 02:00:00+00:00,5
4,2021-08-17 02:00:00+00:00,4
...,...,...
556771,2021-03-29 07:00:00+00:00,1
556772,2021-03-29 12:00:00+00:00,2
556773,2021-03-29 16:00:00+00:00,200
556774,2021-03-29 17:00:00+00:00,4


In [213]:
df = df.groupby('Date et heure de comptage', as_index=False)['Nombre de véhicules'].sum()
df = df.rename(columns={'Date et heure de comptage': 'date', 'Nombre de véhicules': 'total_traffic'})

df['date'] = df['date'].dt.tz_convert(None) # removing the time zone lement of the date for the merge later on

In [214]:
df.shape

(8951, 2)

In [215]:
df.head()

,date,total_traffic
0,2020-08-31 21:00:00,654
1,2020-08-31 22:00:00,492
2,2020-08-31 23:00:00,293
3,2020-09-01 00:00:00,185
4,2020-09-01 01:00:00,116


## **<u>Dropping the first three rows that don't belong to the desired date range:</u>**

In [217]:
df = df.drop([0,1,2], axis=0).reset_index()
df = df.drop('index', axis=1)

In [221]:
df

,date,total_traffic
0,2020-09-01 00:00:00,185
1,2020-09-01 01:00:00,116
2,2020-09-01 02:00:00,104
3,2020-09-01 03:00:00,210
4,2020-09-01 04:00:00,408
...,...,...
8943,2021-09-09 16:00:00,14255
8944,2021-09-09 17:00:00,13251
8945,2021-09-09 18:00:00,10223
8946,2021-09-09 19:00:00,7508


In [219]:
df.to_csv("total_traffic_v1.csv", index=False)